<a href="https://colab.research.google.com/github/cavs1010/personal-projects/blob/main/GC_Real_Estate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Data Collection

The data will be collected from https://www.domain.com.au/rent/gold-coast-qld/ <br> I tried using www.realestate.com.au but there is not permission.

In [27]:
# Install the main libraries for web scrapping
! pip3 install requests
! pip3 install beautifulsoup4

In [28]:
#Import Main Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import math

In [29]:
URL = 'https://www.domain.com.au/rent/gold-coast-qld/'
page = requests.get(URL)

In [30]:
soup = BeautifulSoup(page.content, 'html.parser')

In [31]:
#Number of web pages to read
numberOfWebsToRead = soup.find("h1", attrs={"class": "css-ekkwk0"}).text
numberOfWebsToRead = int(re.findall(r'\d+', numberOfWebsToRead)[0])
numberOfWebsToRead = numberOfWebsToRead /21
numberOfWebsToRead = math.floor(numberOfWebsToRead)

In [32]:
 #Arrays for the ammenities in the house
 mainAddressArray = []
 secondAddressArray = []
 postalCodeArray = []
 rentArray = []
 bedsArray = []
 bathsArray = []
 parkingArray = []
 buildingTypeArray = []

for i in range (1,7):
  URL = 'https://www.domain.com.au/rent/gold-coast-qld/?page='+str(i)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  #Each element will cotain the info required for my work
  elements = soup.find_all("div", class_="css-1n74r2t")
  for element in elements:
    mainAddress = element.find("span", attrs={"data-testid": "address-line1"})
    secondAddress = element.find("span", attrs={"itemprop": "addressLocality"})
    postalCode = element.find("span", attrs={"itemprop": "postalCode"})
    rent = element.find("p", attrs={"data-testid": "listing-card-price"})
    ammenities = element.find_all("span", attrs={"class": "css-1rzse3v"})
    buildingType = element.find("span", attrs={"class": "css-693528"})
    if all(label is not None for label in [mainAddress, secondAddress, postalCode, rent, ammenities[0], ammenities[1], ammenities[2], buildingType]):
      mainAddressArray.append(mainAddress.text)
      secondAddressArray.append(secondAddress.text)
      postalCodeArray.append(postalCode.text)
      rentArray.append(rent.text)
      bedsArray.append(ammenities[0].text)
      bathsArray.append(ammenities[1].text)
      parkingArray.append(ammenities[2].text)
      buildingTypeArray.append(buildingType.text)
    else:
      print('It was not possible to add an element in page '+str(i)+" becasue it's missing some values")

It was not possible to add an element in page 3 becasue it's missing some values


## Final data gathered:

In [33]:
dictionary = {'Main Address': mainAddressArray, 
              'Second Address': secondAddressArray, 
              'Postal Code': postalCodeArray, 
              'Rent':rentArray, 
              'Bedrooms': bedsArray, 
              'Bathrooms': bathsArray,
              'Parking spaces': parkingArray,
              'Building Type': buildingTypeArray}
df = pd.DataFrame(dictionary)
df

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"53 Oxford St,",PIMPAMA,4209,$480 Weekly,4 .css-9fxapx{position:absolute;width:1px;heig...,2 Baths,2 Parking,House
1,"1 Macarthur Street,",PIMPAMA,4209,$490 Per Week,4 Beds,1 Bath,2 Parking,House
2,"55 Nicola Way,",UPPER COOMERA,4209,$500 per week,4 Beds,2 Baths,2 Parking,House
3,"27 Florabella Drive,",ROBINA,4226,$600,3 Beds,2 Baths,2 Parking,House
4,"1/18 Lenneberg Street,",SOUTHPORT,4215,$650,3 Beds,2 Baths,2 Parking,Townhouse
...,...,...,...,...,...,...,...,...
94,"17/120 The Esplanade,",SURFERS PARADISE,4217,$380 P/W Ocean View!,2 Beds,1 Bath,1 Parking,Apartment / Unit / Flat
95,"26 Springsure Drive,",MUDGEERABA,4213,$650,3 Beds,2 Baths,2 Parking,House
96,"19 Norfolk Avenue,",SURFERS PARADISE,4217,$600 per week,3 Beds,2 Baths,2 Parking,House
97,"3/57 Garrick Street,",COOLANGATTA,4225,$485,2 Beds,1 Bath,1 Parking,Apartment / Unit / Flat


# 2. Data Cleaning

In [34]:
dfCopy = df.copy()

In [35]:
#Function that transform the columns[Bedrooms, Bathrooms and Parking Spaces] to integers
#columnName : Name of the column to transform into an integer.
def toInteger (columnName):
  for ind in dfCopy[columnName].index:
    value = dfCopy[columnName][ind][0]
    if (value == '−'):
      value = 0
    else:
      value = int(value)
    dfCopy[columnName][ind] = value

In [36]:
toInteger('Bathrooms')
toInteger('Bedrooms')
toInteger('Parking spaces')
dfCopy

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"53 Oxford St,",PIMPAMA,4209,$480 Weekly,4,2,2,House
1,"1 Macarthur Street,",PIMPAMA,4209,$490 Per Week,4,1,2,House
2,"55 Nicola Way,",UPPER COOMERA,4209,$500 per week,4,2,2,House
3,"27 Florabella Drive,",ROBINA,4226,$600,3,2,2,House
4,"1/18 Lenneberg Street,",SOUTHPORT,4215,$650,3,2,2,Townhouse
...,...,...,...,...,...,...,...,...
94,"17/120 The Esplanade,",SURFERS PARADISE,4217,$380 P/W Ocean View!,2,1,1,Apartment / Unit / Flat
95,"26 Springsure Drive,",MUDGEERABA,4213,$650,3,2,2,House
96,"19 Norfolk Avenue,",SURFERS PARADISE,4217,$600 per week,3,2,2,House
97,"3/57 Garrick Street,",COOLANGATTA,4225,$485,2,1,1,Apartment / Unit / Flat


## Transform the rent into a float:

In [37]:

rentIntArray = []
for ind in dfCopy['Rent'].index:
  #Extract the numbers from the string
  lista = re.findall(r'\d+', dfCopy['Rent'][ind])
  finalRent = ''.join(lista)
  #Eliminate the row if the number is ''
  if (finalRent == ''):
    dfCopy.drop([ind], inplace=True)
    print ('This is the problem index: '+str(ind)+', so it was eliminated' )
  #Otherwise, it add the number to a list
  else:
    dfCopy['Rent'][ind] = float(finalRent)
    rentIntArray.append(float(finalRent))

This is the problem index: 50, so it was eliminated


In [39]:
dfCopy.head()

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type
0,"53 Oxford St,",PIMPAMA,4209,480,4,2,2,House
1,"1 Macarthur Street,",PIMPAMA,4209,490,4,1,2,House
2,"55 Nicola Way,",UPPER COOMERA,4209,500,4,2,2,House
3,"27 Florabella Drive,",ROBINA,4226,600,3,2,2,House
4,"1/18 Lenneberg Street,",SOUTHPORT,4215,650,3,2,2,Townhouse


## Get the latitude and Longitude

In [47]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")


In [48]:
housesNotFound = []
for index, row in dfCopy.iterrows():
  if '/' in row['Main Address']:
    location =  geolocator.geocode(row['Main Address'].split('/',1)[1] +row['Second Address']) 
  else:
    location = geolocator.geocode(row['Main Address']+row['Second Address']) 
  if location == None:
    housesNotFound.append( row['Main Address']+row['Second Address'])
    dfCopy.drop([index], inplace=True)
  else:
    dfCopy['Latitude'] = location.latitude
    dfCopy['Longitude'] = location.longitude

print('These houses were not found:'+ str(housesNotFound))

These houses were not found:[]


In [46]:
housesNotFound

[]

In [49]:
dfCopy

,Main Address,Second Address,Postal Code,Rent,Bedrooms,Bathrooms,Parking spaces,Building Type,Latitude,Longitude
1,"1 Macarthur Street,",PIMPAMA,4209,490,4,1,2,House,53.645831,-2.993097
2,"55 Nicola Way,",UPPER COOMERA,4209,500,4,2,2,House,53.645831,-2.993097
3,"27 Florabella Drive,",ROBINA,4226,600,3,2,2,House,53.645831,-2.993097
4,"1/18 Lenneberg Street,",SOUTHPORT,4215,650,3,2,2,Townhouse,53.645831,-2.993097
5,"5/3 Central St,",LABRADOR,4215,350,1,1,1,Apartment / Unit / Flat,53.645831,-2.993097
...,...,...,...,...,...,...,...,...,...,...
94,"17/120 The Esplanade,",SURFERS PARADISE,4217,380,2,1,1,Apartment / Unit / Flat,53.645831,-2.993097
95,"26 Springsure Drive,",MUDGEERABA,4213,650,3,2,2,House,53.645831,-2.993097
96,"19 Norfolk Avenue,",SURFERS PARADISE,4217,600,3,2,2,House,53.645831,-2.993097
97,"3/57 Garrick Street,",COOLANGATTA,4225,485,2,1,1,Apartment / Unit / Flat,53.645831,-2.993097


### 3. Data Analysis

In [50]:
! pip install folium

In [52]:
import folium

In [61]:
goldCoastMap = folium.Map(location = [-28.0167, 153.4000],  
                          zoom_start = 10,
                          tiles="CartoDB positron")

In [62]:
goldCoastMap